# Script 02 — Decisions, Plans, and Actions

This script introduces three capabilities that turn a "thinking" agent
into one that can **decide**, **plan**, and **act**:

- **DECIDE** — constrained choice from a finite set of options
- **PLAN** — structured multi-step execution plans
- **ACTION primitives** — elemental verbs for interacting with the world

By the end you will be able to build agents that classify inputs,
create plans, search the web, read/write files, and call functions.

Prerequisites:
  - Groq API key in `.env`
  - Brave Search API key in `.env`
  - `pip install thoughtflow`

In [1]:
from _setup import load_env, print_heading, print_separator
import os

env = load_env()

from thoughtflow import (
    LLM, MEMORY, THOUGHT,
    DECIDE, PLAN,
    SEARCH, FETCH, ASK, CALL, READ, WRITE, NOOP,
)

# We will use Groq for all LLM calls in this script.
llm = LLM("groq:llama-3.3-70b-versatile", key=os.environ.get("GROQ_API_KEY", ""))

---
## Part 1 — DECIDE

`DECIDE` is a THOUGHT subclass that constrains the LLM's response to
one of a fixed set of choices.  It handles parsing, validation, and
retry automatically.  If the LLM wanders off-script, DECIDE re-prompts
until it gets a valid answer (up to `max_retries` attempts).

In [2]:
print_heading("1.1  Sentiment classification (list of choices)")

mem = MEMORY()
mem.set_var("review", (
    "The battery life is incredible and the screen is gorgeous, "
    "but the camera quality in low light is disappointing."
))

classify_sentiment = DECIDE(
    name="sentiment",
    llm=llm,
    choices=["positive", "negative", "mixed"],
    prompt="Classify the overall sentiment of this product review: {review}",
)

mem = classify_sentiment(mem)

print("Review  :", mem.get_var("review")[:80], "...")
print("Sentiment:", mem.get_var("sentiment_result"))


══════════════════════════════════════════════════════════════════════
  1.1  Sentiment classification (list of choices)
══════════════════════════════════════════════════════════════════════

Review  : The battery life is incredible and the screen is gorgeous, but the camera qualit ...
Sentiment: mixed


In [3]:
print_heading("1.2  Intent routing (dict of choices with descriptions)")

# When you pass a dict, the descriptions are shown to the LLM to help
# it understand each option.

mem = MEMORY()
mem.set_var("message", "I was charged twice for my last order and I want a refund.")

route = DECIDE(
    name="route",
    llm=llm,
    choices={
        "billing": "Payment issues, refunds, charges, invoices",
        "shipping": "Delivery status, tracking, lost packages",
        "product": "Product quality, features, compatibility",
        "account": "Login, password, profile, settings",
        "general": "Everything else",
    },
    prompt="Route this customer message to the correct department: {message}",
)

mem = route(mem)

print("Message:", mem.get_var("message"))
print("Routed to:", mem.get_var("route_result"))


══════════════════════════════════════════════════════════════════════
  1.2  Intent routing (dict of choices with descriptions)
══════════════════════════════════════════════════════════════════════

Message: I was charged twice for my last order and I want a refund.
Routed to: billing


In [4]:
print_heading("1.3  DECIDE with a default fallback")

# If the LLM fails to choose after all retries, DECIDE can fall back
# to a default value instead of returning None.

mem = MEMORY()
mem.set_var("input_text", "sdkjfhsdkjfh random gibberish 12345")

safe_classify = DECIDE(
    name="priority",
    llm=llm,
    choices=["urgent", "normal", "low"],
    prompt="Classify the priority level of: {input_text}",
    default="normal",
    max_retries=2,
)

mem = safe_classify(mem)
print("Input :", mem.get_var("input_text"))
print("Priority:", mem.get_var("priority_result"))


══════════════════════════════════════════════════════════════════════
  1.3  DECIDE with a default fallback
══════════════════════════════════════════════════════════════════════

Input : sdkjfhsdkjfh random gibberish 12345
Priority: low


---
## Part 2 — PLAN

`PLAN` is a THOUGHT subclass that generates structured, multi-step
execution plans.  You define the **available actions** (with optional
parameter schemas), and the LLM produces a plan as:

```
[ [step_0_tasks], [step_1_tasks], ... ]
```

- Outer list: steps executed sequentially
- Inner list: tasks that can run in parallel within a step
- Each task: `{"action": "...", "params": {...}, "reason": "..."}`

PLAN validates the output structure, action names, and required params.

In [5]:
print_heading("2.1  Generate a research plan")

mem = MEMORY()
mem.set_var("goal", "Understand the current state of quantum computing in 2026")

research_planner = PLAN(
    name="research_plan",
    llm=llm,
    actions={
        "search": {
            "description": "Search the web for information",
            "params": {"query": "str"},
        },
        "fetch": {
            "description": "Fetch and read a specific URL",
            "params": {"url": "str"},
        },
        "analyze": {
            "description": "Analyze gathered information for key insights",
            "params": {"focus": "str"},
        },
        "summarize": {
            "description": "Create a synthesis of findings",
            "params": {"format": "str?"},
        },
    },
    prompt="Create a research plan to: {goal}",
    max_steps=5,
    max_parallel=3,
)

mem = research_planner(mem)
plan = mem.get_var("research_plan_result")

print("Generated plan ({} steps):".format(len(plan)))
for step_idx, step in enumerate(plan):
    print("\n  Step {}:".format(step_idx))
    for task in step:
        params_str = ", ".join("{}={}".format(k, v) for k, v in task.get("params", {}).items())
        print("    - {} ({})".format(task["action"], params_str))
        print("      Reason: {}".format(task.get("reason", "")))


══════════════════════════════════════════════════════════════════════
  2.1  Generate a research plan
══════════════════════════════════════════════════════════════════════

Generated plan (5 steps):

  Step 0:
    - search (query=quantum computing 2026)
      Reason: Start by gathering relevant information on the current state of quantum computing in 2026.

  Step 1:
    - fetch (url=https://www ieee.org/publications/quantum-computing)
      Reason: Fetch information from a reputable source to get in-depth knowledge on quantum computing.

  Step 2:
    - analyze (focus={step_0_result})
      Reason: Analyze search results to extract key insights on the current state of quantum computing.

  Step 3:
    - analyze (focus={step_1_result})
      Reason: Analyze fetched information to get a deeper understanding of quantum computing.

  Step 4:
    - summarize ()
      Reason: Condense findings into a synthesis of the current state of quantum computing in 2026.


In [6]:
print_heading("2.2  Plan with simple action descriptions")

# You can also pass actions as simple strings (no parameter schemas).

mem = MEMORY()
mem.set_var("goal", "Prepare a 3-course dinner for two people")

dinner_planner = PLAN(
    name="dinner_plan",
    llm=llm,
    actions={
        "search_recipe": "Find a recipe online",
        "check_pantry": "Check what ingredients are available",
        "shop": "Buy missing ingredients",
        "prep": "Prepare and measure ingredients",
        "cook": "Cook a dish",
        "plate": "Plate and serve",
    },
    prompt="Plan the steps to: {goal}",
    max_steps=6,
    max_parallel=3,
    validate_params=False,
)

mem = dinner_planner(mem)
plan = mem.get_var("dinner_plan_result")

print("Dinner plan ({} steps):".format(len(plan)))
for step_idx, step in enumerate(plan):
    tasks_summary = " | ".join(t["action"] for t in step)
    print("  Step {}: {}".format(step_idx, tasks_summary))


══════════════════════════════════════════════════════════════════════
  2.2  Plan with simple action descriptions
══════════════════════════════════════════════════════════════════════

Dinner plan (6 steps):
  Step 0: search_recipe
  Step 1: check_pantry
  Step 2: shop
  Step 3: prep
  Step 4: cook
  Step 5: plate


---
## Part 3 — ACTION Primitives

Actions are the agent's **verbs** — they interact with the outside world.
Every action follows the same pattern:

```python
memory = action(memory)
result = memory.get_var("{name}_result")  # or the configured store_as
```

We will demonstrate 7 of the 14 built-in action primitives.

In [7]:
print_heading("3.1  SEARCH — web search with Brave")

# SEARCH supports DuckDuckGo (free), Brave, and EXA.
# We use Brave here for high-quality results.

mem = MEMORY()

search = SEARCH(
    name="web_search",
    provider="brave",
    query="thoughtflow python library agent framework",
    api_key=os.environ.get("BRAVE_API_KEY", ""),
    max_results=5,
)

mem = search(mem)
results = mem.get_var("web_search_results")

print("Query   :", results.get("query"))
print("Provider:", results.get("provider"))
print("Results ({}):" .format(len(results.get("results", []))))
for r in results.get("results", []):
    print("  {}. {} — {}".format(r["rank"], r["title"][:60], r["url"][:50]))


══════════════════════════════════════════════════════════════════════
  3.1  SEARCH — web search with Brave
══════════════════════════════════════════════════════════════════════

Query   : thoughtflow python library agent framework
Provider: brave
Results (5):
  1. Top 7 Python Frameworks for AI Agents - KDnuggets — https://www.kdnuggets.com/top-7-python-frameworks-
  2. Python agent framework — Build powerful agents with ease — https://www.agno.com/agent-framework
  3. thoughtflow · PyPI — https://pypi.org/project/thoughtflow/
  4. GitHub - PrefectHQ/ControlFlow: 🦾 Take control of your AI ag — https://github.com/PrefectHQ/ControlFlow
  5. GitHub - microsoft/agent-framework: A framework for building — https://github.com/microsoft/agent-framework


In [8]:
print_heading("3.2  SEARCH with variable substitution")

# The query can reference memory variables using {variable} placeholders.

mem = MEMORY()
mem.set_var("topic", "event sourcing patterns")

dynamic_search = SEARCH(
    name="topic_search",
    provider="brave",
    query="{topic} best practices 2026",
    api_key=os.environ.get("BRAVE_API_KEY", ""),
    max_results=3,
)

mem = dynamic_search(mem)
results = mem.get_var("topic_search_results")

print("Resolved query:", results.get("query"))
for r in results.get("results", []):
    print("  - {}".format(r["title"][:70]))


══════════════════════════════════════════════════════════════════════
  3.2  SEARCH with variable substitution
══════════════════════════════════════════════════════════════════════

Resolved query: event sourcing patterns best practices 2026
  - Microservices Pattern: Pattern: Event sourcing
  - Event Sourcing pattern - Azure Architecture Center | Microsoft Learn
  - Event Sourcing


In [9]:
print_heading("3.3  FETCH — HTTP requests")

# FETCH makes HTTP requests and parses the response.

mem = MEMORY()

fetch_api = FETCH(
    name="joke",
    url="https://official-joke-api.appspot.com/random_joke",
    method="GET",
    parse="json",
)

mem = fetch_api(mem)
response = mem.get_var("joke_response")

if response.get("success"):
    data = response.get("data", {})
    print("Setup    :", data.get("setup"))
    print("Punchline:", data.get("punchline"))
else:
    print("Fetch result:", response)


══════════════════════════════════════════════════════════════════════
  3.3  FETCH — HTTP requests
══════════════════════════════════════════════════════════════════════

Setup    : Want to hear a chimney joke?
Punchline: Got stacks of em! First one's on the house


In [10]:
print_heading("3.4  CALL — invoke any Python function")

# CALL wraps any callable.  Parameters can reference memory variables.

def fibonacci(n):
    """Compute the nth Fibonacci number."""
    n = int(n)
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

mem = MEMORY()
mem.set_var("fib_index", 10)

call_fib = CALL(
    name="fibonacci",
    function=fibonacci,
    params={"n": "{fib_index}"},
)

mem = call_fib(mem)
print("fibonacci(10) =", mem.get_var("fibonacci_result"))


══════════════════════════════════════════════════════════════════════
  3.4  CALL — invoke any Python function
══════════════════════════════════════════════════════════════════════

fibonacci(10) = 55


In [11]:
print_heading("3.5  CALL — wrapping an external function")

# A more practical example: a function that processes data.

import json as json_module

def extract_emails(text):
    """Pull email addresses from a block of text."""
    import re
    return re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', text)

mem = MEMORY()
mem.set_var("raw_text", (
    "Contact us at support@example.com or sales@example.com. "
    "For press inquiries: press@example.org"
))

extract = CALL(
    name="extract_emails",
    function=extract_emails,
    params={"text": "{raw_text}"},
)

mem = extract(mem)
print("Found emails:", mem.get_var("extract_emails_result"))


══════════════════════════════════════════════════════════════════════
  3.5  CALL — wrapping an external function
══════════════════════════════════════════════════════════════════════

Found emails: ['support@example.com', 'sales@example.com.', 'press@example.org']


In [12]:
print_heading("3.6  WRITE and READ — file I/O")

import tempfile, os

# WRITE saves content to a file.
tmp_dir = tempfile.mkdtemp()
output_path = os.path.join(tmp_dir, "demo_output.json")

mem = MEMORY()
mem.set_var("analysis", {
    "topic": "event sourcing",
    "keywords": ["CQRS", "event store", "projection", "replay"],
    "confidence": 0.92,
})

write_action = WRITE(
    name="save_analysis",
    path=output_path,
    content=lambda m: m.get_var("analysis"),
    mode="json",
)

mem = write_action(mem)
write_result = mem.get_var("save_analysis_result")
print("Wrote {} bytes to {}".format(
    write_result.get("bytes_written"),
    write_result.get("path"),
))


══════════════════════════════════════════════════════════════════════
  3.6  WRITE and READ — file I/O
══════════════════════════════════════════════════════════════════════

Wrote 136 bytes to /var/folders/vy/hdl322p54t5fg8ftxtl6hdl40000gn/T/tmpsm6kj41y/demo_output.json


In [13]:
# READ loads content back from a file.

read_action = READ(
    name="load_analysis",
    path=output_path,
    parse="json",
    store_as="loaded_data",
)

mem = read_action(mem)
loaded = mem.get_var("loaded_data")
print("Read back:")
print("  topic:", loaded.get("topic"))
print("  keywords:", loaded.get("keywords"))
print("  confidence:", loaded.get("confidence"))

# Clean up
import shutil
shutil.rmtree(tmp_dir)

Read back:
  topic: event sourcing
  keywords: ['CQRS', 'event store', 'projection', 'replay']
  confidence: 0.92


In [14]:
print_heading("3.7  ASK — prompt the user for input")

# ASK uses input() under the hood.  In a notebook, this creates a
# text input field.  Here we demonstrate it in a non-blocking way
# by showing how it works — you can uncomment the execution line
# to try it interactively.

ask_name = ASK(
    name="get_name",
    prompt="What is your name?",
    store_as="user_name",
    default="World",
    timeout=10,
)

print("ASK is configured:", ask_name)
print("(Skipping interactive input in this script — see Script 03 for CHAT.)")
print("Default value would be: '{}'".format(ask_name.default))

# To run interactively, uncomment:
# mem = MEMORY()
# mem = ask_name(mem)
# print("Hello,", mem.get_var("user_name"))


══════════════════════════════════════════════════════════════════════
  3.7  ASK — prompt the user for input
══════════════════════════════════════════════════════════════════════

ASK is configured: ASK: What is your name? -> user_name
(Skipping interactive input in this script — see Script 03 for CHAT.)
Default value would be: 'World'


In [15]:
print_heading("3.8  NOOP — explicit no-operation")

# NOOP is useful as a placeholder or for conditional pipelines.

mem = MEMORY()
feature_enabled = False

# Instead of if/else around the whole pipeline, use NOOP.
action = NOOP(reason="Feature not yet implemented") if not feature_enabled else search

mem = action(mem)
result = mem.get_var("noop_result")
print("NOOP result:", result)


══════════════════════════════════════════════════════════════════════
  3.8  NOOP — explicit no-operation
══════════════════════════════════════════════════════════════════════

NOOP result: {'status': 'noop', 'reason': 'Feature not yet implemented'}


---
## Putting It Together — A Mini Pipeline

Let us chain DECIDE + SEARCH + THOUGHT into a small pipeline that:
1. Decides what kind of query the user has
2. Searches the web for relevant information
3. Synthesizes an answer

In [16]:
print_heading("Mini pipeline: Decide → Search → Synthesize")

mem = MEMORY()
mem.set_var("user_question", "What are the main differences between REST and GraphQL?")

# Step 1: Decide the query type.
classify_query = DECIDE(
    name="query_type",
    llm=llm,
    choices={
        "factual": "A question with a definitive answer",
        "comparison": "Comparing two or more things",
        "opinion": "Seeking subjective views or recommendations",
        "howto": "Looking for step-by-step instructions",
    },
    prompt="Classify this question: {user_question}",
)
mem = classify_query(mem)
print("Query type:", mem.get_var("query_type_result"))

# Step 2: Search the web.
web_search = SEARCH(
    name="research",
    provider="brave",
    query="{user_question}",
    api_key=os.environ.get("BRAVE_API_KEY", ""),
    max_results=5,
)
mem = web_search(mem)

search_results = mem.get_var("research_results")
snippets = "\n".join(
    "- {}: {}".format(r["title"], r["snippet"][:120])
    for r in search_results.get("results", [])
)
mem.set_var("search_snippets", snippets)
print("Found {} search results".format(len(search_results.get("results", []))))

# Step 3: Synthesize an answer.
synthesize = THOUGHT(
    name="answer",
    llm=llm,
    prompt=(
        "The user asked: {user_question}\n\n"
        "Query type: {query_type_result}\n\n"
        "Here are relevant search snippets:\n{search_snippets}\n\n"
        "Provide a clear, well-structured answer.  "
        "If this is a comparison, use a side-by-side format."
    ),
    required_vars=["user_question", "query_type_result", "search_snippets"],
)
mem = synthesize(mem)

print("\nSynthesized answer:")
print_separator()
print(mem.get_var("answer_result"))
print_separator()


══════════════════════════════════════════════════════════════════════
  Mini pipeline: Decide → Search → Synthesize
══════════════════════════════════════════════════════════════════════

Query type: comparison
Found 5 search results

Synthesized answer:
──────────────────────────────────────────────────────────────────────
**REST vs GraphQL: A Comparison of API Design Architectures**

When it comes to designing APIs, two popular architectures are REST (Representational State of Resource) and GraphQL. While both have their own strengths and weaknesses, they differ significantly in their approach to data retrieval and manipulation.

**Overview**
|  | REST | GraphQL |
| --- | --- | --- |
| **Introduction** | A widely used, resource-based architecture | A query language for APIs, introduced by Facebook |
| **Data Retrieval** | Client requests specific resources, server returns fixed data | Client requests specific data, server returns only requested data |
| **Flexibility** | Less flexi

---
## Recap

| Primitive | Purpose | Key pattern |
|-----------|---------|-------------|
| **DECIDE** | Constrained choice from fixed options | `mem = decide(mem)` → one of N choices |
| **PLAN** | Structured multi-step execution plan | `mem = plan(mem)` → `[[tasks], [tasks], ...]` |
| **SEARCH** | Web search (DuckDuckGo / Brave / EXA) | `mem = search(mem)` → normalized results |
| **FETCH** | HTTP requests | `mem = fetch(mem)` → response data |
| **CALL** | Invoke any Python function | `mem = call(mem)` → function result |
| **READ/WRITE** | File I/O | `mem = read(mem)` / `mem = write(mem)` |
| **ASK** | Prompt user for input | `mem = ask(mem)` → user's response |
| **NOOP** | Explicit no-operation | `mem = noop(mem)` → `{status: "noop"}` |

Next: **Script 03** — Building a conversational research agent.